# Estudios Python
Este desarrollo se encuentra sobre python 3.7.0

## Contenidos:
Dataset y Kaggle
### 1.- [Iteración Sobre Dataset usando Ctrl+Enter](#section_id1)
### 2.- [Combinación de Cadena de Metodos](#section_id2)
### 3.- [Uso de DataFrameMapper, libreria de Sklearn_pandas](#section_id3)
### 4.- [Uso de TQDM libreria para ProgressBar](#section_id4)

### Kaggle y dataset
Para el uso de este ejemplo se utiliza la libreria de Kaggle (API) para obtener la base del titanic, descomprimirla y posteriormente utilizarla como dataframe

In [54]:
#!pip install kaggle

In [57]:
!kaggle competitions download -c titanic


  0%|          | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████| 34.1k/34.1k [00:00<00:00, 3.16MB/s]


In [58]:
import zipfile as zf 
files = zf.ZipFile("titanic.zip",'r')
files.extractall('dataset')
files.close()

In [59]:
! del titanic.zip

In [60]:
import pandas as pd 
df = pd.read_csv('dataset/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [61]:
print(df.columns)
print(df.info())

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


<a id='section_id1'></a>
## 1.- Analisis de muestras sobre un Dataframe con df.groupby().__iter__()

Objetivo:<br>
Usualmente es complicado explorar un conjunto de datos linea por linea, Si uno compara con Excel. Un truco bastante bueno es utilizar ctrl + Enter (comando) de manera de iterar una vista diferente en cada acción del comando.

Vamos a crear un generador con groupby() <br>
generador = df.groupby(['identificador'])."__iter__"

Posteriormente vamos a seleccionar el Grupo y vamos a iterar sobre este grupo.<br>
grupo_id, grupo_data = generador."__next()__"

se imprime el Grupo y los Datos


In [62]:
tickets_compartidos_con_2_o_mas = df['Ticket'].value_counts().loc[
    (df['Ticket'].value_counts()>2)
    ].index.to_list()


In [63]:
# vamos a crear el generador para poder iterar a traves de lo tickets
generator = df.loc[df['Ticket'].isin(tickets_compartidos_con_2_o_mas)].groupby('Ticket').__iter__()


### En esta Sección es la que puedes ir iterando el dataset con *Ctrl+Enter* en cada grupo

In [70]:
# Se mira a de manera individual por cada Grupo usando control + Enter
group_id, group_data = generator.__next__()
print(group_id)
group_data

17421


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
306,307,1,1,"Fleming, Miss. Margaret",female,NaN,0,0,17421,110.8833,NaN,C
550,551,1,1,"Thayer, Mr. John Borland Jr",male,17.0,0,2,17421,110.8833,C70,C
581,582,1,1,"Thayer, Mrs. John Borland (Marian Longstreth M...",female,39.0,1,1,17421,110.8833,C68,C
698,699,0,1,"Thayer, Mr. John Borland",male,49.0,1,1,17421,110.8833,C68,C


<a id='section_id2'></a>
## 2.- Combinación de multiples operaciones usando una cadena de metodos
Ejemplo de cadean de metodos: <br>
1.- Agregar una nueva columns (.merge) <br>
2.- contar la proporcion de mujeres pasajeras (.apply(female_proportion)) <br>
3.- Agrupar uno o mas pasajeros con cantidad de tickets (df.ticket.value_counts()>1) <br>
4.- obtener el mismo n° ticket 

In [74]:
def female_proportion(dataframe):
    return (dataframe.Sex =='female')/len(dataframe)

female_proportion(df)

0      0.000000
1      0.001122
2      0.001122
3      0.001122
4      0.000000
         ...   
886    0.000000
887    0.001122
888    0.001122
889    0.000000
890    0.000000
Name: Sex, Length: 891, dtype: float64

In [75]:
df.merge(
    # la creacion de female proportion se une con el dataframe segun el nro de ticket
    df.loc[ #se seleccionan todos los ticket mayores a uno que tiene valores y se agrupan por ticket, reseteado el indice 
        df.Ticket.isin( #se obtiene solo un True si tiene un valor
            df.Ticket.value_counts().loc[ #filtra todos los tickets que tienen mas que 1 y entrega el indice (index['347082', '1601', 'CA. 2343', 'CA 2144', '3101295', '347088'...])
                df.Ticket.value_counts()>1 #entrega un Bool si es mayor que 1
            ].index
        )
    ].groupby('Ticket').apply(female_proportion) \
    .reset_index().rename(columns={0:'proportion_female'}) #  reseteado el indice  y cambiando el nombre a la columnta 0
    ,how='left', on='Ticket'
)


,PassengerId,Survived,Pclass,Name,Sex_x,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,level_1,Sex_y
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,NaN,NaN
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,3.0,0.5
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,137.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,NaN,NaN
1589,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,783.0,0.0
1590,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,888.0,0.5
1591,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,NaN,NaN


<a id='section_id3'></a>
## 3.- Uso de DataFramMapper correspondiente a la libreria Sklearn_pandas
Objetivo

usualmente cuando se desea crear un modelo se debe preprocesar la información (categorias, standarizacion,etc..) para ello se utiliza [sklearn preprocesing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) y utilizar hacer un mix de estas librerias puede ser engorrozo. Para ello es donde viene la a participar una amigable libreria DataFramMapper el cual es el Reemplazo de ColumnTransforme.

Para ello vamos a crear un pipeline de transformación Imputacion, Standarizacion y Categorización el cual vamos a utilizar en el Dataframe.

In [26]:
#!pip install scikit-learn
#!pip install sklearn
#!pip install sklearn_pandas
#!pip install category-encoders

In [27]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn_pandas import DataFrameMapper
from category_encoders import LeaveOneOutEncoder

In [28]:
imputer_Pclass = SimpleImputer(strategy='most_frequent', add_indicator=True)
imputer_Age = SimpleImputer(strategy='median', add_indicator=True)
imputer_SibSp = SimpleImputer(strategy='constant', fill_value=0, add_indicator=True)
imputer_Parch = SimpleImputer(strategy='constant', fill_value=0, add_indicator=True)
imputer_Fare = SimpleImputer(strategy='median', add_indicator=True)
imputer_Embarked = SimpleImputer(strategy='most_frequent')

In [29]:
scaler_Age = MinMaxScaler()
scaler_Fare = StandardScaler()

In [30]:
onehotencoder_Sex = OneHotEncoder(drop=['male'], handle_unknown='error')
onehotencoder_Embarked = OneHotEncoder(handle_unknown='error')
leaveoneout_encoder = LeaveOneOutEncoder(sigma=.1, random_state=2020)

In [34]:
mapper = DataFrameMapper([
    (['Age'], [imputer_Age, scaler_Age], {'alias':'Age_scaled'}),
    (['Pclass'], [imputer_Pclass]),
    (['SibSp'], [imputer_SibSp]),
    (['Parch'], [imputer_Parch]),
    (['Fare'], [imputer_Fare, scaler_Fare], {'alias': 'Fare_scaled'}),
    (['Sex'], [onehotencoder_Sex], {'alias': 'is_female'}),
    (['Embarked'], [imputer_Embarked, onehotencoder_Embarked])
], df_out=True) # use df_out to output as a pandas DataFrame

In [35]:
mapper.fit(X=df, y=df['Survived']) # you fit it like a sklearn ColumnTransformer

DataFrameMapper(default=False, df_out=True,
                features=[(['Age'],
                           [SimpleImputer(add_indicator=True, copy=True,
                                          fill_value=None, missing_values=nan,
                                          strategy='median', verbose=0),
                            MinMaxScaler(copy=True, feature_range=(0, 1))],
                           {'alias': 'Age_scaled'}),
                          (['Pclass'],
                           [SimpleImputer(add_indicator=True, copy=True,
                                          fill_value=None, missing_values=nan,
                                          strategy='most_freque...
                                          dtype=<class 'numpy.float64'>,
                                          handle_unknown='error',
                                          sparse=True)],
                           {'alias': 'is_female'}),
                          (['Embarked'],
                   

In [36]:
mapper.fit_transform(df)

,Age_scaled_0,Age_scaled_1,Pclass,SibSp,Parch,Fare_scaled,is_female,Embarked_x0_C,Embarked_x0_Q,Embarked_x0_S
0,0.271174,0.0,3,1,0,-0.502445,0.0,0.0,0.0,1.0
1,0.472229,0.0,1,1,0,0.786845,1.0,1.0,0.0,0.0
2,0.321438,0.0,3,0,0,-0.488854,1.0,0.0,0.0,1.0
3,0.434531,0.0,1,1,0,0.420730,1.0,0.0,0.0,1.0
4,0.434531,0.0,3,0,0,-0.486337,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
886,0.334004,0.0,2,0,0,-0.386671,0.0,0.0,0.0,1.0
887,0.233476,0.0,1,0,0,-0.044381,1.0,0.0,0.0,1.0
888,0.346569,1.0,3,1,2,-0.176263,1.0,0.0,0.0,1.0
889,0.321438,0.0,1,0,0,-0.044381,0.0,1.0,0.0,0.0


<a id='section_id4'></a>
## 4.  Uso de la libreria [TQDM](https://tqdm.github.io/)

### corresponde al uso de una barra de desarrollo o barra de progreso. 

In [42]:
#!pip install tqdm

In [43]:
from tqdm import notebook
notebook.tqdm().pandas()

C:\Users\rvill\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [46]:
from tqdm import trange
import time

for i in trange(100):
    time.sleep(0.01)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 95.17it/s]


In [47]:
from tqdm import tqdm
from time import sleep

text = ""
for char in tqdm(["a", "b", "c", "d"]):
    sleep(0.25)
    text = text + char

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.98it/s]


In [48]:
pbar = tqdm(["a", "b", "c", "d"])
for char in pbar:
    sleep(0.25)
    pbar.set_description("Processing %s" % char)

Processing d: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.98it/s]


In [49]:
with tqdm(total=100) as pbar:
    for i in range(10):
        sleep(0.1)
        pbar.update(10)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 99.25it/s]


In [51]:
pbar = tqdm(total=100)
for i in range(10):
    sleep(0.5)
    pbar.update(10)
pbar.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.97it/s]
